In [1]:
from keras.layers import Input, Dense, Concatenate,Lambda,Reshape,Dropout, BatchNormalization, Conv2DTranspose, UpSampling2D
from keras.models import Model
import numpy as np


vector = Input(shape=(512,), name='Vectors-Input',dtype='float32')
vectorReshaped = Reshape((64,8,1),name='Vectors-Input-Reshaped')(vector)
deconv1 = Conv2DTranspose(64, (10,2), strides=(1,1),name='Deconvolution1')(vectorReshaped)
norm_deconv1=BatchNormalization()(deconv1)
deconv2 = Conv2DTranspose(64, (12,3), strides=(1,1), name='Deconvolution2')(norm_deconv1)
norm_deconv2=BatchNormalization()(deconv2)
deconv3 = Conv2DTranspose(60, (12,4), strides=(1,1),name='Deconvolution3')(norm_deconv2)
norm_deconv3=BatchNormalization()(deconv3)
deconv4 = Conv2DTranspose(60, (12,4), strides=(1,1),name='Deconvolution4')(norm_deconv3)
norm_deconv4=BatchNormalization()(deconv4)
deconv5 = Conv2DTranspose(55, (12,5), strides=(1,1),name='Deconvolution5')(norm_deconv4)
norm_deconv5=BatchNormalization()(deconv5)
deconv6 = Conv2DTranspose(55, (12,5), strides=(1,1),name='Deconvolution6')(norm_deconv5)
norm_deconv6=BatchNormalization()(deconv6)
deconv7 = Conv2DTranspose(32, (6,6), strides=(2,2),padding='same',name='Deconvolution7')(norm_deconv6)
norm_deconv7=BatchNormalization()(deconv7)
deconv8 = Conv2DTranspose(1, (8,8), strides=(2,2),padding='same',name='Deconvolution8')(norm_deconv7)
output = Reshape((512,100),name='Output-Reshaped')(deconv8)


model = Model(inputs=vector,outputs=output)
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
)
model.summary()

Using TensorFlow backend.
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu








Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Vectors-Input (InputLayer)   (None, 512)               0         
_________________________________________________________________
Vectors-Input-Reshaped (Resh (None, 64, 8, 1)          0         
_________________________________________________________________
Deconvolution1 (Conv2DTransp (None, 73, 9, 64)         1344      
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 9, 64)         256       
_________________________________________________________________
Deconvolution2 (Conv2DTransp (None, 84, 11, 64)        147520    
_________________________________________________________________
batch_normalization_2 (Batch (None, 84, 11, 64)        256       
_________________________________________________________________
Deconvolution3 (Conv2DTransp (None, 95, 14, 60)     

In [6]:
from bpemb import BPEmb
from collections import Iterable
import numpy as np

bpemb_ru = BPEmb(lang="ru", dim=100, vs=3000)

def textreader(st,limit):
    texts = []
    with open("D:/112/decoder/data/data3000/np_rev2.txt", "r", encoding='utf-8') as fr:
        i=0
        for line in fr:
            i+=1
            if i<st:      
                continue
            texts.append(line.replace('\n',''))
            if i==st+limit-1:
                break
    return texts

def word_vectors_creator(data):
    l=len(data)
    kol= np.zeros(shape=(l))
    pieces=[]
    for i in range(l):
        s = bpemb_ru.encode_ids(data[i])
        kol[i]+=len(s)
        pieces.append(s)
    kol = kol.astype(np.int)
    out = np.zeros(shape=(l,512,100))
    for g in range(l):
        out[g][kol[g]] = np.ones(100)
        for k in range(1,kol[g]-1):
            out[g][k+1]+=bpemb_ru.emb.vectors[pieces[g][k]]
        out[g][kol[g]+2:]-= np.ones(shape=(512-kol[g]-2,100))
        out[g][kol[g]+2:]-= np.ones(shape=(512-kol[g]-2,100))    
    return out, kol       #, out2

In [7]:
from sklearn.utils import shuffle


n=0
e=1
for d in range(2):
    start=1
    print('Epoch: '+str(d)+'\n'+"**********************************\n")
    for i in range(800):
        vectors_train = np.load('D:/112/decoder/data/data3000/news_vectors'+str(999-i)+'.npy')
        #print(start)
        texts = textreader(start,vectors_train.shape[0])
        start+=vectors_train.shape[0]
        #print(len(texts))

        #for j in range(5):
        tt, kol  = word_vectors_creator(texts)
        #vtt = vectors_train[0+j*100:(1+j)*100]
        tt, vectors_train = shuffle(tt, vectors_train, random_state=0)

        if n%20==0:
            model.fit(
                x=vectors_train,
                y =tt,
                epochs=e, initial_epoch=n,
                validation_split=0.05,
                verbose=1,
                batch_size=8
            )
            if n%500==0:
                model.save('D:/112/decoder/models/conv_mse_sen_to_word'+str(d)+'_'+str(i)+'.h5')
                print('\n')
        else:
            model.fit(
                x=vectors_train,
                y =tt,
                epochs=e, initial_epoch=n,
                validation_split=0.05,
                verbose=0,
                batch_size=8
            )
        n+=1
        e+=1
    model.save('D:/112/decoder/models/conv_mse_sen_to_word'+str(d)+'.h5')
    print('\n\n\n')

Epoch: 0
**********************************

Train on 475 samples, validate on 25 samples
Epoch 1/1
475/475 [==============================] - 8s 17ms/step - loss: 2.1809 - val_loss: 0.9535


Train on 475 samples, validate on 25 samples
Epoch 21/21
475/475 [==============================] - 2s 5ms/step - loss: 0.1452 - val_loss: 0.1232
Train on 475 samples, validate on 25 samples
Epoch 41/41
475/475 [==============================] - 2s 5ms/step - loss: 0.1688 - val_loss: 0.1674
Train on 475 samples, validate on 25 samples
Epoch 61/61
475/475 [==============================] - 2s 5ms/step - loss: 0.1828 - val_loss: 0.1512
Train on 475 samples, validate on 25 samples
Epoch 81/81
475/475 [==============================] - 3s 5ms/step - loss: 0.1660 - val_loss: 0.1159
Train on 475 samples, validate on 25 samples
Epoch 101/101
475/475 [==============================] - 3s 5ms/step - loss: 0.1575 - val_loss: 0.2285
Train on 475 samples, validate on 25 samples
Epoch 121/121
475/475 [========

475/475 [==============================] - 2s 5ms/step - loss: 0.1886 - val_loss: 0.2301
Train on 475 samples, validate on 25 samples
Epoch 1101/1101
475/475 [==============================] - 2s 5ms/step - loss: 0.1476 - val_loss: 0.1659
Train on 475 samples, validate on 25 samples
Epoch 1121/1121
475/475 [==============================] - 2s 5ms/step - loss: 0.1943 - val_loss: 0.1167
Train on 475 samples, validate on 25 samples
Epoch 1141/1141
475/475 [==============================] - 2s 5ms/step - loss: 0.1694 - val_loss: 0.1925
Train on 475 samples, validate on 25 samples
Epoch 1161/1161
475/475 [==============================] - 2s 5ms/step - loss: 0.1753 - val_loss: 0.1477
Train on 475 samples, validate on 25 samples
Epoch 1181/1181
475/475 [==============================] - 2s 5ms/step - loss: 0.1585 - val_loss: 0.1476
Train on 474 samples, validate on 25 samples
Epoch 1201/1201
474/474 [==============================] - 2s 5ms/step - loss: 0.1787 - val_loss: 0.1817
Train on 47